In [1]:
import numpy as np
import pandas as pd
from starfish.plot import *

In [2]:
from starfish.set import settings
settings.cosmo_value

[0.32, 0.049, 0.67, 0.96, 0.81, 0.06, 3.044]

In [3]:
def var_filter(fisher,lcdm,neff_fixed):
    df=fisher.copy()
    if lcdm:
        df=df.drop('w0',axis=0)
        df=df.drop('w0',axis=1)
        df=df.drop('wa',axis=0)
        df=df.drop('wa',axis=1)
    if neff_fixed:
        df=df.drop('N_eff',axis=0)
        df=df.drop('N_eff',axis=1)

    return df

# nuisance parameters of different tracers/surveys should be independent of each other

In [4]:
JPAS_multi=var_filter(pd.read_excel('../data/jpas_all/valid.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=False)
cov=pd.DataFrame(np.linalg.inv(JPAS_multi.values),index=JPAS_multi.index,columns=JPAS_multi.columns)
JPAS_multi_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

In [5]:
PFS_ELG=var_filter(pd.read_excel('../data/PFS/fisher.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=False)
cov=pd.DataFrame(np.linalg.inv(PFS_ELG.values),index=PFS_ELG.index,columns=PFS_ELG.columns)
PFS_ELG_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

# J-PAS multitracer + PFS ELG

In [6]:
fisher1=pd.DataFrame(JPAS_multi_valid+PFS_ELG_valid,index=settings.var_name,columns=settings.var_name)
fisher1.to_excel('../data/combined/pelg+jmul-lcdm-free.xlsx')
fisher1

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,543831.929977,-9.858576e+05,74888.073197,221829.070537,-10062.167937,-22067.404750,-8468.205857
omega_b0,-985857.647127,2.505464e+06,-410464.011839,-423082.660153,44839.019531,42500.389546,21380.711460
h,74888.073197,-4.104640e+05,285152.296991,48556.944450,-98803.121008,-5392.025981,-4066.114502
n_s,221829.070537,-4.230827e+05,48556.944450,103838.218472,-17728.695611,-10113.712011,-3841.449518
sigma_8,-10062.167937,4.483902e+04,-98803.121008,-17728.695611,82575.267173,3933.386348,457.330952
m_nu,-22067.404750,4.250039e+04,-5392.025981,-10113.712011,3933.386348,1160.792168,371.743250
N_eff,-8468.205857,2.138071e+04,-4066.114502,-3841.449518,457.330952,371.743250,198.359135


In [7]:
cov1=pd.DataFrame(np.linalg.inv(fisher1.values),index=settings.var_name,columns=settings.var_name)
cov1

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,0.000036,0.000003,-0.000038,-0.000036,-0.000088,0.001005,-0.001902
omega_b0,0.000003,0.000007,0.000008,-0.000012,0.000017,-0.000230,-0.000353
h,-0.000038,0.000008,0.000133,0.000010,0.000259,-0.002846,0.005131
n_s,-0.000036,-0.000012,0.000010,0.000157,0.000016,0.000418,0.002202
sigma_8,-0.000088,0.000017,0.000259,0.000016,0.000534,-0.005938,0.009989
m_nu,0.001005,-0.000230,-0.002846,0.000418,-0.005938,0.072424,-0.104611
N_eff,-0.001902,-0.000353,0.005131,0.002202,0.009989,-0.104611,0.282681


In [8]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov1.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 6.0e-03
-- omega_b0: 2.7e-03
--        h: 1.2e-02
--      n_s: 1.3e-02
--  sigma_8: 2.3e-02
--     m_nu: 2.7e-01
--    N_eff: 5.3e-01


# + CMB-S4 + LiteBIRD

In [9]:
cmb_fisher=var_filter(pd.read_excel('../data/CMB/all.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=False)
cmb_fisher

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,tau
omega_m0,1.851288e+08,-6.928981e+08,1.371057e+08,-4.740936e+06,-5.782434e+07,-1.244715e+07,-6.589565e+06,4.485034e+07
omega_b0,-6.928981e+08,3.023402e+09,-4.583542e+08,3.206098e+07,2.226094e+08,4.714503e+07,2.328230e+07,-1.714219e+08
h,1.371057e+08,-4.583542e+08,1.125702e+08,-1.894718e+06,-3.817385e+07,-7.840066e+06,-5.484485e+06,2.962660e+07
n_s,-4.740936e+06,3.206098e+07,-1.894718e+06,1.046895e+06,1.665063e+06,3.149959e+05,1.125701e+05,-1.284039e+06
sigma_8,-5.782434e+07,2.226094e+08,-3.817385e+07,1.665063e+06,2.236167e+07,5.252308e+06,1.591679e+06,-1.731189e+07
m_nu,-1.244715e+07,4.714503e+07,-7.840066e+06,3.149959e+05,5.252308e+06,1.282975e+06,2.946820e+05,-4.090356e+06
N_eff,-6.589565e+06,2.328230e+07,-5.484485e+06,1.125701e+05,1.591679e+06,2.946820e+05,2.884944e+05,-1.230550e+06
tau,4.485034e+07,-1.714219e+08,2.962660e+07,-1.284039e+06,-1.731189e+07,-4.090356e+06,-1.230550e+06,1.374361e+07


In [10]:
cmb_cov=pd.DataFrame(np.linalg.inv(cmb_fisher.values),index=cmb_fisher.index,columns=cmb_fisher.columns)
cmb_cov_marg=cmb_cov.drop('tau',axis=0)
cmb_cov_marg=cmb_cov_marg.drop('tau',axis=1)
cmb_fisher_marg=pd.DataFrame(np.linalg.inv(cmb_cov_marg),index=settings.var_name,columns=settings.var_name)
cmb_fisher_marg

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,3.876607e+07,-1.334868e+08,4.042346e+07,-5.506558e+05,-1.329430e+06,9.011528e+05,-2.573839e+06
omega_b0,-1.334868e+08,8.852823e+08,-8.882606e+07,1.604536e+07,6.680925e+06,-3.873349e+06,7.933847e+06
h,4.042346e+07,-8.882606e+07,4.870520e+07,8.732384e+05,-8.552429e+05,9.773665e+05,-2.831834e+06
n_s,-5.506558e+05,1.604536e+07,8.732384e+05,9.269296e+05,4.764654e+04,-6.715809e+04,-2.397812e+03
sigma_8,-1.329430e+06,6.680925e+06,-8.552429e+05,4.764654e+04,5.550676e+05,9.996606e+04,4.163926e+04
m_nu,9.011528e+05,-3.873349e+06,9.773665e+05,-6.715809e+04,9.996606e+04,6.560846e+04,-7.155261e+04
N_eff,-2.573839e+06,7.933847e+06,-2.831834e+06,-2.397812e+03,4.163926e+04,-7.155261e+04,1.783157e+05


In [11]:
fisher1_combined=fisher1+cmb_fisher_marg
fisher1_combined.to_excel('../data/combined/cmb+pelg+jmul-lcdm-free.xlsx')
cov1_combined=pd.DataFrame(np.linalg.inv(fisher1_combined),index=fisher1_combined.index,columns=fisher1_combined.columns)
cov1_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,4.413188e-06,4.610228e-07,-2.853084e-06,-7.731881e-07,-4.943057e-06,0.000027,0.000010
omega_b0,4.610228e-07,8.021343e-08,-5.709149e-07,-3.483369e-07,-8.705789e-07,0.000003,-0.000005
h,-2.853084e-06,-5.709149e-07,4.502682e-06,2.623976e-06,6.096699e-06,-0.000016,0.000048
n_s,-7.731881e-07,-3.483369e-07,2.623976e-06,4.277045e-06,1.787033e-06,0.000005,0.000048
sigma_8,-4.943057e-06,-8.705789e-07,6.096699e-06,1.787033e-06,1.589020e-05,-0.000055,0.000038
m_nu,2.735459e-05,2.836389e-06,-1.607698e-05,5.113445e-06,-5.535282e-05,0.000300,0.000147
N_eff,1.001597e-05,-4.651170e-06,4.794424e-05,4.779041e-05,3.838887e-05,0.000147,0.001171


In [12]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov1_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 2.1e-03
-- omega_b0: 2.8e-04
--        h: 2.1e-03
--      n_s: 2.1e-03
--  sigma_8: 4.0e-03
--     m_nu: 1.7e-02
--    N_eff: 3.4e-02


# CMB-S4 + LiteBIRD + PFS

In [13]:
PFS_combined=PFS_ELG_valid+cmb_fisher_marg
PFS_combined.to_excel('../data/combined/cmb+pelg-lcdm-free.xlsx')
cov_combined=pd.DataFrame(np.linalg.inv(PFS_combined),index=PFS_combined.index,columns=PFS_combined.columns)
cov_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,0.000021,2.135319e-06,-0.000015,-2.524124e-06,-0.000018,1.046939e-04,0.000016
omega_b0,0.000002,2.771597e-07,-0.000002,-7.671800e-07,-0.000003,1.093923e-05,-0.000008
h,-0.000015,-1.986305e-06,0.000015,5.906697e-06,0.000018,-7.262681e-05,0.000078
n_s,-0.000003,-7.671800e-07,0.000006,7.173953e-06,0.000005,-7.674435e-07,0.000090
sigma_8,-0.000018,-2.544412e-06,0.000018,5.133534e-06,0.000033,-1.314761e-04,0.000074
m_nu,0.000105,1.093923e-05,-0.000073,-7.674435e-07,-0.000131,7.111921e-04,0.000187
N_eff,0.000016,-8.086171e-06,0.000078,9.012264e-05,0.000074,1.870017e-04,0.001885


In [14]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 4.5e-03
-- omega_b0: 5.3e-04
--        h: 3.8e-03
--      n_s: 2.7e-03
--  sigma_8: 5.8e-03
--     m_nu: 2.7e-02
--    N_eff: 4.3e-02
